In [1]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil

# --- CONFIGURACIÓN ---
# Rutas del modelo y el dataset
YOLO5_REPO_PATH = 'yolov5'
MODEL_PATH = 'best.pt'
DATASET_BASE_PATH = 'yolo_dataset'
TEST_SUBSET_NAME = 'val'  # Usaremos el conjunto 'val' como nuestro conjunto de test

# Parámetros del análisis
IOU_THRESHOLD = 0.1
NUM_IMAGES_TO_TEST = 5000  # Número de imágenes a procesar

# Nombres de las carpetas de salida
OUTPUT_DIR_BARCOS = 'barcos'
OUTPUT_DIR_ERRORES = 'misclassifications'

# --- FUNCIONES DE AYUDA ---

def calculate_iou(boxA, boxB):
    """Calcula la Intersección sobre Unión (IoU) entre dos recuadros [xmin, ymin, xmax, ymax]."""
    interArea = max(0, min(boxA[2], boxB[2]) - max(boxA[0], boxB[0])) * max(0, min(boxA[3], boxB[3]) - max(boxA[1], boxB[1]))
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = float(boxAArea + boxBArea - interArea)
    return interArea / unionArea if unionArea > 0 else 0

def load_ground_truth_boxes(annotation_path, img_width, img_height):
    """Carga los recuadros desde un archivo .txt de YOLO y los desnormaliza."""
    boxes = []
    if not os.path.exists(annotation_path): return boxes
    with open(annotation_path, 'r') as f:
        for line in f:
            _, x_center, y_center, width, height = map(float, line.strip().split())
            w_abs, h_abs = width * img_width, height * img_height
            xmin = (x_center * img_width) - (w_abs / 2)
            ymin = (y_center * img_height) - (h_abs / 2)
            xmax = xmin + w_abs
            ymax = ymin + h_abs
            boxes.append([xmin, ymin, xmax, ymax])
    return boxes

# --- FUNCIÓN PRINCIPAL DEL SCRIPT ---

def main():
    """
    Función principal que carga el modelo, procesa las imágenes y guarda los recortes sin mostrar ninguna GUI.
    """
    # --- 1. Cargar el modelo ---
    print("Cargando el modelo YOLOv5...")
    model = torch.hub.load(YOLO5_REPO_PATH, 'custom', path=MODEL_PATH, source='local', force_reload=True)
    if torch.cuda.is_available():
        model.to('cuda')
        print("Modelo cargado en la GPU.")
    else:
        print("Modelo cargado en la CPU.")

    # --- 2. Preparar carpetas de salida ---
    print(f"Creando/limpiando carpetas de salida: '{OUTPUT_DIR_BARCOS}' y '{OUTPUT_DIR_ERRORES}'")
    for dir_path in [OUTPUT_DIR_BARCOS, OUTPUT_DIR_ERRORES]:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
        os.makedirs(dir_path)

    # --- 3. Obtener la lista de imágenes a procesar ---
    images_path = os.path.join(DATASET_BASE_PATH, TEST_SUBSET_NAME, 'images')
    labels_path = os.path.join(DATASET_BASE_PATH, TEST_SUBSET_NAME, 'labels')
    if not os.path.exists(images_path):
        print(f"❌ Error fatal: No se encuentra el directorio de imágenes en '{images_path}'")
        return
    test_images = sorted(os.listdir(images_path))[:NUM_IMAGES_TO_TEST]
    print(f"Se procesarán las primeras {len(test_images)} imágenes de '{images_path}'")

    # --- 4. Bucle principal de procesamiento ---
    for image_name in tqdm(test_images, desc="Procesando imágenes"):
        image_path = os.path.join(images_path, image_name)
        label_path = os.path.join(labels_path, os.path.splitext(image_name)[0] + '.txt')

        original_image = cv2.imread(image_path)
        if original_image is None: continue
        
        img_height, img_width, _ = original_image.shape
        gt_boxes = load_ground_truth_boxes(label_path, img_width, img_height)

        img_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        results = model(img_rgb)
        pred_df = results.pandas().xyxy[0]
        pred_boxes = pred_df[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()

        # --- Lógica de matching y guardado de archivos ---
        gt_boxes_matched = [False] * len(gt_boxes)
        base_filename = os.path.splitext(image_name)[0]

        # Procesar predicciones (TP y FP)
        for i, pred_box in enumerate(pred_boxes):
            xmin, ymin, xmax, ymax = map(int, pred_box)
            crop_img = original_image[ymin:ymax, xmin:xmax]
            if crop_img.size == 0: continue

            best_iou, best_gt_idx = 0, -1
            for j, gt_box in enumerate(gt_boxes):
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou: best_iou, best_gt_idx = iou, j
            
            if best_iou > IOU_THRESHOLD and best_gt_idx != -1 and not gt_boxes_matched[best_gt_idx]:
                # VERDADERO POSITIVO (TP)
                save_path = os.path.join(OUTPUT_DIR_BARCOS, f"{base_filename}_tp_{i}.png")
                cv2.imwrite(save_path, crop_img)
                gt_boxes_matched[best_gt_idx] = True
            else:
                # FALSO POSITIVO (FP)
                save_path = os.path.join(OUTPUT_DIR_ERRORES, f"{base_filename}_fp_{i}.png")
                cv2.imwrite(save_path, crop_img)

        # Procesar verdad terrenal no detectada (FN)
        for i, matched in enumerate(gt_boxes_matched):
            if not matched:
                xmin, ymin, xmax, ymax = map(int, gt_boxes[i])
                crop_img = original_image[ymin:ymax, xmin:xmax]
                if crop_img.size == 0: continue
                
                save_path = os.path.join(OUTPUT_DIR_BARCOS, f"{base_filename}_fn_{i}.png")
                cv2.imwrite(save_path, crop_img)
    
    # --- 5. Finalización ---
    print("\n¡Proceso completado!")
    print(f"Los recortes de los barcos se han guardado en la carpeta '{OUTPUT_DIR_BARCOS}'.")
    print(f"Los recortes de las detecciones incorrectas se han guardado en la carpeta '{OUTPUT_DIR_ERRORES}'.")


# --- Punto de entrada del script ---
if __name__ == "__main__":
    main()

Cargando el modelo YOLOv5...


c:\Users\angel\Desktop\Carpetas\SpaceChallenges\project\HRSID_JPG\PruebaClasificacion\yolov5\utils\general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
YOLOv5  v7.0-422-g2540fd4c Python-3.13.1 torch-2.7.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Modelo cargado en la CPU.
Creando/limpiando carpetas de salida: 'barcos' y 'misclassifications'
Se procesarán las primeras 1962 imágenes de 'yolo_dataset\val\images'


Procesando imágenes:   0%|          | 0/1962 [00:00<?, ?it/s]c:\Users\angel\Desktop\Carpetas\SpaceChallenges\project\HRSID_JPG\PruebaClasificacion\yolov5\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Procesando imágenes:   0%|          | 1/1962 [00:00<23:05,  1.42it/s]c:\Users\angel\Desktop\Carpetas\SpaceChallenges\project\HRSID_JPG\PruebaClasificacion\yolov5\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Procesando imágenes:   0%|          | 2/1962 [00:01<15:34,  2.10it/s]c:\Users\angel\Desktop\Carpetas\SpaceChallenges\project\HRSID_JPG\PruebaClasificacion\yolov5\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autoc


¡Proceso completado!
Los recortes de los barcos se han guardado en la carpeta 'barcos'.
Los recortes de las detecciones incorrectas se han guardado en la carpeta 'misclassifications'.
